In [15]:
from rcnn.load_data import rcnn_format

(train_x, train_y), (test_x, test_y) = rcnn_format('Face', (92, 112, 3))  #彩色時(寬,高,cheanl)

print(train_x.shape)
print(test_x.shape)

Load dataset: Face\pos
(40, 112, 92, 3)
(10, 112, 92, 3)


In [16]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
import numpy as np

In [17]:
#自訂網路架構
model = Sequential()

model.add(Conv2D(filters=8, kernel_size=(5, 5), padding='same', activation='relu', input_shape=(112, 92, 3))) #彩色時(高,寬,cheanl)
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=8, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=8, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

#model.add(Conv2D(filters=8, kernel_size=(3, 3), padding='same', activation='relu'))
#model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))

In [18]:
model.summary()

# 卷積層 Param =（卷積核長度 *卷積核寬度 *通道數 +1）*卷積核個數
# 全連接層 Param = （輸入數據維度+1）* 神經元個數
# 加1是考慮到每個神經元都有一個 Bias

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 112, 92, 8)        608       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 56, 46, 8)        0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 56, 46, 8)         584       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 28, 23, 8)        0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 28, 23, 8)         584       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 14, 11, 8)       

In [19]:
model_layers = [Model(inputs=model.layers[0].input, outputs=model.layers[0].output),
                Model(inputs=model.layers[0].input, outputs=model.layers[1].output),
                Model(inputs=model.layers[0].input, outputs=model.layers[3].output),
                Model(inputs=model.layers[0].input, outputs=model.layers[5].output),
                #Model(inputs=model.layers[0].input, outputs=model.layers[7].output),
               ]
print(model.layers[0].input)
print(model.layers[1].output)
print(model.layers[3].output)
print(model.layers[5].output)
#print(model.layers[7].output)

KerasTensor(type_spec=TensorSpec(shape=(None, 112, 92, 3), dtype=tf.float32, name='conv2d_3_input'), name='conv2d_3_input', description="created by layer 'conv2d_3_input'")
KerasTensor(type_spec=TensorSpec(shape=(None, 56, 46, 8), dtype=tf.float32, name=None), name='max_pooling2d_3/MaxPool:0', description="created by layer 'max_pooling2d_3'")
KerasTensor(type_spec=TensorSpec(shape=(None, 28, 23, 8), dtype=tf.float32, name=None), name='max_pooling2d_4/MaxPool:0', description="created by layer 'max_pooling2d_4'")
KerasTensor(type_spec=TensorSpec(shape=(None, 14, 11, 8), dtype=tf.float32, name=None), name='max_pooling2d_5/MaxPool:0', description="created by layer 'max_pooling2d_5'")


In [20]:
%matplotlib inline
from ipywidgets import interact
import matplotlib.pyplot as plt
from matplotlib import patches

def interact_result(data):
    preds = []
    for model_layer in model_layers:
        preds.append(model_layer.predict(data))
    
    def view_image(i):
        # 層數要改
        fig, axes = plt.subplots(4, 8, figsize = (25, 25)) 
        fig.subplots_adjust(hspace=0.01, wspace=0.4)
        for l, pred in enumerate(preds):  
            print(pred.shape)
            for n in range(pred[i].shape[2]):
                axes[l,n].imshow(pred[i][:,:,n] * 255)
        
    interact(view_image, i=(0, data.shape[0]-1))

In [21]:
interact_result(test_x)

1/1 [==============================] - 0s 130ms/step


interactive(children=(IntSlider(value=4, description='i', max=9), Output()), _dom_classes=('widget-interact',)…

In [22]:
interact_result(train_x)

2/2 [==============================] - 0s 15ms/step


interactive(children=(IntSlider(value=19, description='i', max=39), Output()), _dom_classes=('widget-interact'…